In [1]:
from source.regency import Regency
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


seasons = 1
Game = Regency(True)
clear_output()
Game.add_relationship('Go','Fh',At_War=1)

#for a in range(seasons):
Game.random_events()
Game.collect_regency_points()
Game.domain_initiative()
Game.collect_gold_bars()
Game.maintenance_costs(Update=True)
#     Game.Seasons[0][1]['Override']['Go'] = (22,'enemy','Fh')
#     Game.take_domain_actions(1)
#     Game.take_domain_actions(2)
#     Game.take_domain_actions(3)
    # Game.clear_screen()

#df = pd.concat([Game.Seasons[b]['Actions'][a+1] for a in range(3) for b in range(seasons)])
#print(set(df['Action']))
#print()
#print(set(df[df['Success?']==True]['Action']))
#Game.War



SyntaxError: invalid syntax (regency.py, line 6025)

In [ ]:
%matplotlib notebook

In [ ]:
Game.Seasons[0]['Override'][1]['JR'] = (22,'enemy','EC')
Game.Seasons[0]['Override'][1]['EC'] = (20,'friend','WM')
Game.Seasons[0]['Override'][1]['WM'] = (21,'friend','EC')
Game.Seasons[0]['Override'][1]['GG'] = (21,'friend','EC')
Game.add_troops('WM','Moergen','Archers')
Game.add_troops('EC','Bogsend','Archers')
Game.add_troops('JR','Bogsend','Archers')
Game.add_troops('WM','Moergen','Archers')
Game.add_troops('EC','Bogsend','Archers')
Game.add_troops('JR','Bogsend','Archers')
Game.add_relationship('JR', 'EC', At_War=1)
Game.add_relationship('WM', 'JR', At_War=1)
Game.add_relationship('GG', 'EC', Diplomacy=3)

In [ ]:
Game.take_domain_actions(1)
Game.War

In [ ]:
print(Game.War.iloc[1]['Notes'])

In [ ]:
Game.Troops[Game.Troops['Provence']==Game.War.iloc[3]['Location']]

In [ ]:
Game.Seasons[0]['Override'][1]['JR'] = (22,'enemy','EC')
Game.Seasons[0]['Override'][1]['EC'] = (20,'friend','WM')
Game.Seasons[0]['Override'][1]['WM'] = (21,'friend','EC')
Game.take_domain_actions(2)

In [ ]:
Game.War

In [ ]:
maps = Mapping(Game)
maps.focus_regions(['The Eastern Marches'])
maps.show(show_abbreviations=True, show_troops=True, borders=True)

In [ ]:
Game.defenders['Provence'].values[0]

In [ ]:
Game.defenders['Home Regent']

In [ ]:
Game.defenders